# Измерение качества моделей

## Метрики качества

Это практическое задание посвящено ознакомлению с инструментами оценки качества моделей машинного обучения, которые предоставляет библиотека `scikit-learn`. Метрики качества, представленные различными функциями, находятся в модуле `sklearn.metrics`. Мы начнем с загрузки набора данных из файла `data.csv` при помощи функции `read_csv` из библиотеки `pandas`. Этот набор данных содержит информацию о предсказаниях различных алгоритмов машинного обучения для решения задачи классификации. Колонка `prediction` - это результаты работы одного из этих алгоритмов. Целевая переменная содержится в столбце `target` (класс 0 или 1). Подсчитайте значение `true negative`, `false negative`, `true positive` и `false positive`. Запишите эти значения через запятую, сохраняя приведенный порядок, в переменную `answer1`, которая будет являтся строкой. Далее, посчитайте для этих данных значение таких метрик как `precision`, `recall` и `f1 score` с точностью до двух знаков после запятой. Запишите результаты в строго заданном порядке через запятую в переменную `answer2`, которая так же будет являться строкой

### *РЕШЕНИЕ*

In [5]:
import pandas as pd
from sklearn.metrics import confusion_matrix
df  = pd.read_csv('data.csv')
tn, fp, fn, tp = confusion_matrix(df.target, df.prediction).ravel()
answer1 = tn, fn, tp, fp
answer1

(63, 4, 114, 7)

In [38]:
from sklearn.metrics import precision_score, recall_score, f1_score
answer2 = round(precision_score(df.target, df.prediction), 2), round(recall_score(df.target, df.prediction), 2), round(f1_score(df.target, df.prediction), 2)
answer2

(0.94, 0.97, 0.95)

В столбцах `scores_1` и `scores_2` содержаться оценки вероятности пренадлежности объектов к классу 1 для двух разных алгоритмов машинного обучения. Рассчитайте площадь под ROC-кривой для каждого алгоритма и сравните их. В качестве ответа `answer3` приведите большее из двух значений, округленное до трех знаков после запятой.

### *РЕШЕНИЕ*

In [11]:
from sklearn.metrics import roc_auc_score
alg1 = roc_auc_score(df.target, df.scores_1)
alg2 = roc_auc_score(df.target, df.scores_2)
answer3 = max(alg1, alg2)
answer3

0.9934624697336562

## Метод скользящего контроля

Во второй части данного практического задания мы изучать различные методы оценки моделей машинного обучения. Загрузите набор данных `Breast Cancer Wisconsin (Diagnostic)`, используя функцию `load_breast_cancer` из модуля `sklearn.datasets`. Этот датасет позволяет решать задачу предсказания рака груди по различным характеристикам опухоли. В данном случае, целевая переменная принимает два значения, соответствующие доброкачественной и злокачественной опухоли. Проверьте, является ли данная выборка сбалансированной.

In [22]:
from sklearn.datasets import load_breast_cancer

X, y = load_breast_cancer(return_X_y=True)
counts = pd.value_counts(y)
print("Is this a balanced dataset? {}".format(counts[1] == counts[0]))
counts

Is this a balanced dataset? False


1    357
0    212
dtype: int64

Первый метод, который мы будем использовать, - это случайное разбиение датасета на тренировочную и тестовую выборку с помощью функции `train_test_split` из `sklearn.model_selection` с параметрами `random_state=3` и `test_size=0.33`. Если выборка является несбалансированной передайте целевую переменную в эту функцию в качестве аргумента `stratify`.

Обучите логистическую регрессию (класс `LogisticRegression` из модуля `sklearn.linear_model`) с параметром конструктора `random_state=42` и метод K ближайших соседей (класс `KNeighborsClassifier` из модуля `sklearn.neighbors`) на тренировочной выборке. Оцените качество на тестовой выборке для каждой из моделей. В качестве метрики качества используйте `recall`. Какая из моделей показывает лучший результат? Ответом на это задание `answer4` является этот результат, округленный до трех знаков после запятой.

### *РЕШЕНИЕ*

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


In [29]:
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=3, test_size=0.33, stratify = cancer.target)
logreg = LogisticRegression(random_state=42)
knereg = KNeighborsClassifier()
logreg.fit(X_train, y_train)
knereg.fit(X_train, y_train)
lpred = logreg.predict(X_test)
knpred = knereg.predict(X_test)
lscore = recall_score(y_test, lpred)
knscore = recall_score(y_test, knpred)
answer4 = max(lscore, knscore)
answer4

C:\Users\margarita.potokina\Anaconda3\envs\coursera_ds\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9745762711864406

Далее мы проведем оценку каждой из этих моделей в соответствии с методом скользящего контроля с помощью функции `cross_val_score` из модуля `sklearn.model_selection`. В качестве параметра кросс-валидации `cv` в этой функции используйте экземпляр класса `StratifiedKFold` из `sklearn.model_selection` с тремя разбиениями. 

Функция `cross_val_score` возвращает количество оценок, соответствующие числу разбиений.
В качестве итогового результата используете среднее значение полученных оценок с помощью метрики `recall`. Какая модель работает лучше в это случае? Какие выводы можно из этого сделать? Ответом на это задание `answer5` является лучший итоговый результат, округленный до трех знаков после запятой.

### *РЕШЕНИЕ*

In [40]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
lcross = cross_val_score(logreg, X, y, cv=StratifiedKFold(n_splits=3), scoring='recall').mean()
kncross = cross_val_score(knereg, X, y, cv=StratifiedKFold(n_splits=3), scoring='recall').mean()
answer5= max(lcross, kncross)
answer5

C:\Users\margarita.potokina\Anaconda3\envs\coursera_ds\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\margarita.potokina\Anaconda3\envs\coursera_ds\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\margarita.potokina\Anaconda3\envs\coursera_ds\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9663865546218489

# Строка с ответами

In [41]:
output = """TN,FN,TP,FP = {0}
Precision,Recall,F1 Score = {1}
Best ROC AUC Score {2:.3f}
Random Split {3:.3f}
Cross Val Score {4:.3f}"""
print(output.format(answer1, answer2, answer3, answer4, answer5))

TN,FN,TP,FP = (63, 4, 114, 7)
Precision,Recall,F1 Score = (0.94, 0.97, 0.95)
Best ROC AUC Score 0.993
Random Split 0.975
Cross Val Score 0.966
